In [1]:
import pandas
import numpy
from scipy.optimize import minimize

In [2]:
motor = 1400 #rpm

table = pandas.read_table("table.txt")
table['ratio'] = table.spindle / motor
table

,spindle,d1,d2,d3,d4,ratio
0,110,45,a,d,184,0.078571
1,190,45,a,c,159,0.135714
2,220,65,b,d,184,0.157143
3,320,45,a,b,136,0.228571
4,380,65,b,c,159,0.271429
5,430,90,c,d,184,0.307143
6,860,65,b,a,117,0.614286
7,1170,90,c,b,136,0.835714
8,1260,99,d,c,159,0.900000
9,1680,90,c,a,117,1.200000


Kiekvieną lentelės eilutę atitinka viena tiesinė lygtis.

$R = \frac{d_1}{d_2} \frac{d_3}{d_4}$

$d_2 = \frac{d_1 d_3}{R d_4}$

$d_2 - \frac{d_1}{R d_4} d_3 = 0$

Fiksuojame A ir sudarome lygčių sistemą su trim nežinomaisiais: B, C, D. Lygčių sistemos matrica:

In [46]:
diameter_a = 154
max_spindle = 2880


def diameters(xs):
    return pandas.Series([diameter_a] + list(xs), index=list('abcd'))

def actual_ratios(ds):
    dsdict = dict(ds)
    return numpy.array([r.d1 / dsdict[r.d2] * dsdict[r.d3] / r.d4 for i, r in table.iterrows()])

def percent_difference(x, y):
    return x / y - 1

def cost(xs):
    speeds = motor * actual_ratios(diameters(xs))
    delta = percent_difference(speeds, table['spindle'])
    delta *= numpy.where(delta > 0, 2, 1)
    cost = numpy.sum(numpy.abs(delta))
    cost += 40 * abs(percent_difference(min(speeds), min(table['spindle'])))
    cost += 50 * abs(max(0, percent_difference(max(speeds), max(table['spindle']))))
    return cost
    
bounds = (45, diameter_a)
def initial():
    return sorted(numpy.random.uniform(*b, 3), reverse=True)
result = minimize(cost, initial(), bounds=(bounds, bounds, bounds))
for i in range(10):
    r = minimize(cost, initial(), bounds=(bounds, bounds, bounds))
    if r.fun < result.fun:
        print(r.fun)
        result = r
    else:
        print('.', end='')
result

13.8246164688
...13.8246164688
.....

      fun: 13.824616468790396
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([  3.55271368e-07,   0.00000000e+00,  -3.55271368e-06])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 92
      nit: 19
   status: 0
  success: True
        x: array([ 125.22054381,   89.90048134,   62.07443187])

In [47]:
ds = diameters(result.x)
ds

a    154.000000
b    125.220544
c     89.900481
d     62.074432
dtype: float64

Gauti dydžiai suapvalinami iki milimetrų:

In [52]:
ds = numpy.round(ds)
ds

a    154
b    125
c     90
d     62
dtype: float64

Gauti greičiai stulpelyje actual_spindle:

In [55]:
table['actual_ratio'] = actual_ratios(ds)
table['actual_spindle'] = table['actual_ratio'] * motor
table

,spindle,d1,d2,d3,d4,ratio,actual_ratio,actual_spindle
0,110,45,a,d,184,0.078571,0.098461,137.845850
1,190,45,a,c,159,0.135714,0.165401,231.560892
2,220,65,b,d,184,0.157143,0.175217,245.304348
3,320,45,a,b,136,0.228571,0.268573,376.002674
4,380,65,b,c,159,0.271429,0.294340,412.075472
5,430,90,c,d,184,0.307143,0.336957,471.739130
6,860,65,b,a,117,0.614286,0.684444,958.222222
7,1170,90,c,b,136,0.835714,0.919118,1286.764706
8,1260,99,d,c,159,0.900000,0.903834,1265.368229
9,1680,90,c,a,117,1.200000,1.316239,1842.735043
